In [1]:
# CELL #1
import matplotlib
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
import scipy
import csv

#import dependency functions

# from utility import (fix_nans, 
#                      del_singles,
#                      del_triples,
#                      del_segment, 
#                      interpolate_traces,
#                      update,
#                      animal_select,
#                      animal_switch_function,
#                      next_segment,
#                      forward_function,
#                      backward_function,
#                      del_jumps_function,
#                      previous_segment,
#                      save,
#                      load_function,
#                      interpolate_function,
#                      export_movie                     
# )


Autosaving every 180 seconds


In [6]:
# CELL #2

def fix_nans(event):
    global tracesx, tracesy

    ids = np.arange(4)
    for id_ in ids:
        for k in range(tracesx.shape[0]):
            if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
                print (k, tracesx[k,id_], tracesy[k,id_])
                tracesx[k,id_]= tracesy[k,id_] =np.nan
                print ('')
                
    print ("DONE FIXING nans")
    update(current_index[0])
    
def del_singles(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-2,1):
            if np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True, 
                                                 False,
                                                 True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan
    print ("DONE DELEting singles")   
    update(current_index[0])
    
    
def del_triples(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-4,1):
            flag_ = False
            if np.all(np.isnan(tracesx[k:k+5,id_])==np.array([True,
                                                              False,False,False,
                                                              True])):
                tracesx[k:k+5,id_]=np.nan
                tracesy[k:k+5,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+4,id_])==np.array([True,
                                                              False,False,
                                                              True])):
                tracesx[k:k+4,id_]=np.nan
                tracesy[k:k+4,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True,
                                                              False,
                                                              True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan 
                flag_ = True
                
            if flag_:
                print ("deleted small segs", id_, k)
    print ("DONE DELEting triples")   
    update(current_index[0])

def del_segment(event):
    global tracesx, tracesy
    
    # search active trace snippet
    # search forward

    def find_continous_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
 
    #end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])

    print ("Selected animal id: ", selected_animal, ", start/end: ", start1, end1)

    # replace switch id info
    tracesx[start1:end1, selected_animal[0]] = np.nan
    tracesy[start1:end1, selected_animal[0]] = np.nan
  
    print ("DONE DELEting triples")   
    update(current_index[0])
    
 

def delete_jumps(tracesx, tracesy):
    #tracesx_ip=tracesx.copy()
    #tracesy_ip=tracesy.copy()
    for k in range(tracesx.shape[1]):

        # for big jumps, set intermediate values to nans
        max_jump = 50
        for p in range(0,tracesx.shape[0]-1,1):
            distx = tracesx[p,k]-tracesx[p+1,k]
            disty = tracesy[p,k]-tracesy[p+1,k]
            dist = np.sqrt(distx**2+disty**2)
            if dist>max_jump:
                #print (snippets[p,0],snippets[p,1])
                tracesx[p:p+2,k]= np.nan
                tracesy[p:p+2,k]= np.nan
                #print ("found large jump: ", p/25.)
            #else:
            #    print (dist, "not merging")


       # break
        #np.isnan(x_loc) or np.isnan(y_loc):
    print ("DONE FIXING JUMPS")
    #print ("tracex: ", tracesx[:100,0])
    
    return tracesx, tracesy


# TODO Add time threshold also; 
def interpolate_traces(tracesx, tracesy, dist_threshold=10., time_threshold=10):
    tracesx_ip=tracesx.copy()
    tracesy_ip=tracesy.copy()
    for k in range(tracesx_ip.shape[1]):
        start = False
        end = False
        idx = np.where(~np.isnan(tracesx_ip[:,k]))[0]

        # make snippets
        snippets=[]
        locs = []
        start = idx[0]
        #for p in range(1,idx.shape[0],1):
        p=1
        ctr=0
        while True:
            if idx[p]-idx[p-1]>1:
                end = idx[p-1]
                snippets.append([start,end])
                start = idx[p]   

                # find locations of start and end
                locs.append([])
                locs[ctr].append([tracesx_ip[start,k], tracesy_ip[start,k]])
                locs[ctr].append([tracesx_ip[end,k], tracesy_ip[end,k]])

                # bump 2 over
                p+=2
                ctr+=1
            else:
                p+=1

            if p>=idx.shape[0]:
                break

        snippets = np.array(snippets)
        #print (snippets.shape)
        #print (snippets)
        locs = np.array(locs)
    #     print (locs.shape)
    #     print (locs[0])

        # fill in space between snippets if euclidean distance < threshold
        #threshold = 10.
        # AND IF NOT TOO FAR APART; 
        for p in range(1, snippets.shape[0]-1):
            # Skip if interpolate across too long a sgement
            if (snippets[p,0] - snippets[p-1,1])>time_threshold:
                continue
            dist = np.linalg.norm(locs[p,0]-locs[p-1,1])
            
            #print ("Dist; ", dist, locs[p,1],locs[p+1,0])
            if dist<dist_threshold:
                print ("interpoalting snippets: ", k, snippets[p,0],snippets[p,1])
                tracesx_ip[snippets[p-1,1]:snippets[p,0],k]= tracesx_ip[snippets[p-1,1],k]
                tracesy_ip[snippets[p-1,1]:snippets[p,0],k]= tracesy_ip[snippets[p-1,1],k]
            #else:
            #    print (dist, "not merging")

    # 
    print ("DELETING JUMPS FUCNTION")
    tracesx, tracesy = delete_jumps(tracesx_ip,tracesy_ip)

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    return tracesx, tracesy


# update the slider
def update(val):
    global tracesx, tracesy

    #amp = samp.val
    index = int(sfreq.val)
    current_index[0] = index
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
    fig.canvas.draw_idle()
    
def advance_function(label):
    
    if label == "ON":
        auto_advance=0
    else:
        auto_advance=1
        
    print ("Auto advance: ", auto_advance)
 
def animal_select(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    update(current_index[0])

def switch_all_screen(event):    

    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH ALL TRACES ON SCREEN BETWEEN 2 ANIMALS")

    label = animal_switch.value_selected
    print ("label: ", label)
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k    
    
    print ("curent animal: ", selected_animal[0], " switch animal = ", k )
    
    start = current_index[0]
    end = current_index[0]*10*25-1  # swap all 10 seconds visible on screen
    
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    # replace switch id info
    tracesx[start:end, switch_id] = tempx[start:end, selected_animal[0]].copy()
    tracesy[start:end, switch_id] = tempy[start:end, selected_animal[0]].copy()
    
    # replace currental animal info
    tracesx[start:end, selected_animal[0]] = tempx[start:end, switch_id].copy()
    tracesy[start:end, selected_animal[0]] = tempy[start:end, switch_id].copy()
    
    # update plots
    update(current_index[0])   
    
    
def animal_swap(label):
    #l.set_color(label)
    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH ALL TRACES BETWEEN 2 ANIMALS FUNCTION")

    label = animal_switch.value_selected
    print ("label: ", label)
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k    
    
    print ("curent animal: ", selected_animal[0], " switch animal = ", k )
    
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    # replace switch id info
    tracesx[:, switch_id] = tempx[:, selected_animal[0]].copy()
    tracesy[:, switch_id] = tempy[:, selected_animal[0]].copy()
    
    # replace currental animal info
    tracesx[:, selected_animal[0]] = tempx[:, switch_id].copy()
    tracesy[:, selected_animal[0]] = tempy[:, switch_id].copy()
    
    # update plots
    update(current_index[0])   
 
def animal_switch_function(label):
    #l.set_color(label)
    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH FUNCTION")

    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k


    # search active trace snippet
    # search forward

    def find_continous_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
    
    print ("switching to : ", labels[switch_id])
    print ("current index inside switch:", current_index)
    
    end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
            
    print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])
    
    print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
        
        
    # DATA SWITCH HAS TO BE SAFE FOR ALL DOWNSTREAM 
    # SO NEED TO CHECK THE ID THAT"S BEING SWAPPED TO MAKE SURE IT DOESN"T GET DELETED;
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    print ("Target switch tracesx: ", tempx[start1:end1, switch_id].copy())
    print ("Selected animal tracesx: ", tempx[start1:end1, selected_animal[0]].copy())

    buff = 2
    print ("Switch ANIMAL TRACES preswitch; animal: ", switch_id,
          tracesx[start1-buff:end1+buff, switch_id],
          tracesy[start1-buff:end1+buff, switch_id])
        

    # replace switch id info
    tracesx[start1:end1, switch_id] = tempx[start1:end1, selected_animal[0]].copy()
    tracesy[start1:end1, switch_id] = tempy[start1:end1, selected_animal[0]].copy()
    
    print ("Switch ANIMAL TRACES postswitch; animal: ", switch_id, 
          tracesx[start1-buff:end1+buff, switch_id],
          tracesy[start1-buff:end1+buff, switch_id])
           
    print ("Selected ANIMAL TRACES preswitch; animal: ",selected_animal[0],
          tracesx[start1-buff:end1+buff, selected_animal[0]],
          tracesy[start1-buff:end1+buff, selected_animal[0]])

    # replace currental animal info
    tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id].copy()
    tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id].copy()
    
    print ("SelectedANIMAL TRACES postswitch; animal ",selected_animal[0],
          tracesx[start1-buff:end1+buff, selected_animal[0]],
          tracesy[start1-buff:end1+buff, selected_animal[0]])
  
    # 
    print ("DELETING JUMPS FUCNTION")
    tracesx, tracesy = delete_jumps(tracesx,tracesy)

    # move to the next snippet
    if auto_advance:
        next_segment(0)
        
    update(current_index[0])   
    

# search for the next continuous chunk of red
def next_segment(event):
    
    # 
    start_nan = n_frames-1
    for k in range(current_index[0], n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)
    
    start_nan+=1
    start_val = n_frames-1
    for k in range(start_nan, n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_val=k
            break

    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = start_val
    sfreq.val = current_index[0]
    fig.canvas.draw_idle()

    print ("..... to : ", current_index[0])
    update(current_index[0])
    
    # search for the next continuous chunk of red

    
def forward_function(event):
    
    # 
    current_index[0]+=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()
    
def backward_function(event):
    
    # 
    current_index[0]-=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()

        
def del_jumps_function(event):
    
    global tracesx, tracesy
    # 
    print ("DELETING JUMPS FUCNTION")
    tracesx, tracesy = delete_jumps(tracesx,tracesy)

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    
def previous_segment(event):
    
    # 
    print ("Current esgemtn: ", tracesx[current_index[0]:current_index[0]+5,
                                        selected_animal[0]])
    
    # END OF PREVIOUS NAN BLOCK
    end_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            end_nan = k
            break
    print ("searching... start_nan: ", end_nan)

    # START OF PREVIOUS NAN BLOCK
    start_nan=0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_nan=k+1
            break

    start_data =0
    for k in range(start_nan-1, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k+1 # select the very next val
            break
            
    print ("Reversing from: ", current_index[0])
    current_index[0] = start_data
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    #fig.canvas.draw_idle()

    
def save(event):
    
    np.savez(traces_saved, 
             tracesx = tracesx,
             tracesy = tracesy)
    print ("SAVING TRACES")
        
def load_function(event):
    
    global tracesx, tracesy
    
    data = np.load(traces_saved)
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    update(current_index[0])

    print ("loaded traces")

def interpolate_function(event):
    global tracesx, tracesy
    
    dist_threshold = 200.
    time_threshold = 25
    tracesx, tracesy = interpolate_traces(tracesx, tracesy, 
                                          dist_threshold,
                                         time_threshold)
    
    update(current_index[0])
    #fig.canvas.draw_idle()
    print ("INTERPOLATING TRACES, THRESHOLD: ", dist_threshold)

def export_movie(movie):
    clrs = ['blue','red','cyan','green']

    size_vid = np.array([1280,1024])
    dot_size = 16
    
    #out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    fname_out = (out_dir+'video_labeled_exported_fixed.avi')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

    # initialize start and ends of video to be analyzed
    start = 0
    #end = start + 10*25
    end = tracesx.shape[0]
    #end = 3000
    original_vid = cv2.VideoCapture(video_name)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    for k in range(start,end,1):    
        if k %100==0:
            print (k)

        ret, frame = original_vid.read()
        if ret==False:
            break
#         if scaling:
#             frame_cropped = frame.copy()[::4,::4]
#         else:
#             frame_cropped = frame.copy()

        # Loop over animals
        for p in range(0, tracesx.shape[1], 1):
            x = tracesx[k,p]
            y = tracesy[k,p]
            # plot large bump
            if np.isnan(x) or np.isnan(y):
                continue
            x = int(x)
            y = int(y)
            
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()
    
    
    
    

In [7]:
# CELL #1
# change flag for windows
linux = True

global current_index, selected_animal, tracesx, tracesy, traces_saved

if linux:
    out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    video_name = os.path.join(out_dir, '2020-3-9_08_18_49_128168_compressed_snippet.avi')
    traces_means = os.path.join(out_dir, 'traces_means_unfiltered.npz') 
    traces_saved = os.path.join(out_dir,'traces_fixed.npz')
    %matplotlib tk

else:
    #out_dir = r"C:\madeline\data\'
    video_name = r"C:\madeline\data\2020-3-9_08_18_49_128168_compressed_snippet.avi"
    traces_name = r"C:\madeline\data\traces_means.npz"
    traces_saved = r"C:\madeline\data\traces_saved.npz"
    %matplotlib qt

    
print (video_name)
data= np.load(traces_means)

tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']
print ("DONE LOADING")

/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi
DONE LOADING


In [8]:
# FUTURE WORK OBJECT ORIENTED PROGRAMMING

# reload data
#tracesx = tracesx_ip.copy() #data['tracesx_mean'].copy()
#tracesy = tracesy_ip.copy() #data['tracesy_mean'].copy()

# class dlc_object():

#     def __init__(self, tracesx, tracesy, traces_saved):
#         self.tracesx=tracesx.copy()
#         self.tracesy=tracesy.copy()        
#         self.traces_saved=traces_saved
        
#         self.current_index = []
#         self.selected_animal = []
#         self.original_vid = []
#         self.n_frames = []

        
        
# dlc = dlc_object(tracesx, tracesy,
#                 traces_saved)
# print (dlc.tracesx.shape)


In [9]:
# CELL 4 LOAD GUI


# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250

# auto advance default setting
auto_advance =[]
auto_advance.append(0)

# save current frame of video
current_index = []
current_index.append(0)
#dlc.current_index=current_index

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)
#dlc.selected_animal=selected_animal

# initialize start and ends of video to be analyzed
frame_rate = 25
original_vid = cv2.VideoCapture(video_name)
#dlc.original_vid = original_vid


# initialize video
loc = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
ret, frame = original_vid.read()
n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

#dlc.n_frames = n_frames


# initialize scatter plots
ax_video = plt.subplot(gs[:, 1:4])
img = ax_video.imshow(frame)
ax_video.set_xlim(150,1200)
ax_video.set_ylim(50,800)

axes_animals=[]
for k in range(4):
    x_loc = tracesx[0,k]
    y_loc = tracesy[0,k]
    if np.isnan(x_loc) or np.isnan(y_loc):
        x_loc=100
        y_loc=100
    axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

# plot traces
ax_tracesx = plt.subplot(gs[:2, :1])
ax_tracesx.set_title("DLC-x")
init_length = 10*25
t = np.arange(init_length)/25.
ax1 = []
for k in range(4):
    temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
    ax1.append(temp)

ax_tracesx.set_xticks([])
ax_tracesx.set_ylim(0,1280)

# plot y traces
ax_tracesy = plt.subplot(gs[2:, :1])
ax_tracesy.set_title("DLC-y")
ax2 = []
for k in range(4):
    temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
    ax2.append(temp)

ax_tracesy.set_xlabel("Time (sec)")
ax_tracesy.set_ylim(0,1024)
# add 
#ax.margins(x=0)

##location of sliders
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
#sfreq = Slider(axfreq, 'Frame',0.20, 18, valinit=0, dragging=True,  fc='blue')    
#sfreq.val=val



sfreq.on_changed(update)
#samp.on_changed(update)

# resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
# button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

# def reset(event):
#     sfreq.reset()
#     #samp.reset()
# button.on_clicked(reset)


# ANIMAL SELECTOR
labels = ["ON", "OFF"]
rax0 = plt.axes([0.01, 0.90, 0.07, 0.1], facecolor=axcolor)
advance_buttons = RadioButtons(rax0, (labels[0],labels[1]), active=0)
rax0.set_title('Auto-advance')
advance_buttons.on_clicked(advance_function)



# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.8, 0.07, 0.1], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
animal_buttons.on_clicked(animal_select)

# ANIMAL SWITCHER
rax2 = plt.axes([0.01, 0.65, 0.07, 0.1], facecolor=axcolor)
animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax2.set_title('Switch') 
animal_switch.on_clicked(animal_switch_function)


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axswitch_all = plt.axes([0.01, 0.6, 0.04, 0.04])
bswitch_all = Button(axswitch_all, 'Switch\nAll')
bswitch_all.on_clicked(switch_all_screen)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axswap = plt.axes([0.06, 0.6, 0.04, 0.04])
bswap = Button(axswap, 'SWAP\nANIMALS')
bswap.on_clicked(animal_swap)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)



    
    
    
#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.01, 0.55, 0.04, 0.04])
bnext = Button(axnext, 'Next\nSegment')
bnext.on_clicked(next_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.01, 0.5, 0.04, 0.04])
bprev = Button(axprev, 'Previous\nSegment')
bprev.on_clicked(previous_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)




#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axforward = plt.axes([0.01, 0.45, 0.04, 0.04])
bforward = Button(axforward, 'Step\nForward')
bforward.on_clicked(forward_function)

# BACK BUTTON
axback = plt.axes([0.01, 0.4, 0.04, 0.04])
bback = Button(axback, 'Step\nBackward')
bback.on_clicked(backward_function)
#bprev = Button(axprev, 'Previous')

# DELETE JUMPS BUTTON
axdel_jumps = plt.axes([0.01, 0.3, 0.04, 0.04])
bdeljumps = Button(axdel_jumps, 'Fix\nJumps')
bdeljumps.on_clicked(del_jumps_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axinterpolate = plt.axes([0.01, 0.25, 0.04, 0.04])
binterpoloate = Button(axinterpolate, 'Interpolate\nTraces')
binterpoloate.on_clicked(interpolate_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axnans = plt.axes([0.01, 0.2, 0.04, 0.04])
bnans = Button(axnans, 'Fix\nNans')
bnans.on_clicked(fix_nans)
#bprev = Button(axprev, 'Previous')



# Delete very short segments automatically
axtriples = plt.axes([0.01, 0.15, 0.04, 0.04])
btriples = Button(axtriples, 'Del\nTriples')
btriples.on_clicked(del_triples)

# Del Segment
axdel_seg = plt.axes([0.06, 0.15, 0.04, 0.04])
bdel_seg = Button(axdel_seg, 'Del\nSegment')
bdel_seg.on_clicked(del_segment)

#bprev = Button(axprev, 'Previous')

# Export movie
axmovie = plt.axes([0.01, 0.05, 0.04, 0.04])
bmovie = Button(axmovie, 'Export\nMovie')
bmovie.on_clicked(export_movie)
#bprev = Button(axprev, 'Previous')

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.06, 0.1, 0.04, 0.04])
bsave = Button(axsave, 'Save\nTraces')
bsave.on_clicked(save)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload = plt.axes([0.06, 0.05, 0.04, 0.04])
bload = Button(axload, 'Load\nTraces')
bload.on_clicked(load_function)

plt.show()

current indx updated: [4038]
current indx updated: [4029]
current indx updated: [4000]
current indx updated: [3877]
current indx updated: [3858]
current indx updated: [4153]
current indx updated: [4410]
DELETING JUMPS FUCNTION
DONE FIXING JUMPS
current indx updated: [4410]
current indx updated: [4400]
current indx updated: [4371]
current indx updated: [4352]
current indx updated: [4295]
current indx updated: [4200]
current indx updated: [4096]
current indx updated: [4086]
current indx updated: [4058]
current indx updated: [3829]
current indx updated: [3629]
current indx updated: [3563]
current indx updated: [3544]
current indx updated: [3534]
current indx updated: [3525]
current indx updated: [3477]
current indx updated: [3363]
current indx updated: [3325]
current indx updated: [3306]
current indx updated: [3239]
current indx updated: [3163]
current indx updated: [3097]
current indx updated: [3087]
current indx updated: [3078]
current indx updated: [3049]
current indx updated: [3030]
c

interpoalting snippets:  2 2219 2221
interpoalting snippets:  2 2239 2243
interpoalting snippets:  2 2247 2248
interpoalting snippets:  2 2255 2263
interpoalting snippets:  2 2267 2284
interpoalting snippets:  2 2288 2311
interpoalting snippets:  2 2318 2325
interpoalting snippets:  2 2333 2386
interpoalting snippets:  2 2390 2403
interpoalting snippets:  2 2407 2421
interpoalting snippets:  2 2467 2543
interpoalting snippets:  2 2748 2749
interpoalting snippets:  2 2752 2756
interpoalting snippets:  2 2761 2771
interpoalting snippets:  2 2808 2828
interpoalting snippets:  2 2837 3091
interpoalting snippets:  2 3094 3101
interpoalting snippets:  2 3105 3109
interpoalting snippets:  2 3116 3117
interpoalting snippets:  2 3125 3152
interpoalting snippets:  2 3172 3183
interpoalting snippets:  2 3192 3198
interpoalting snippets:  2 3219 3242
interpoalting snippets:  2 3246 3252
interpoalting snippets:  2 3255 3256
interpoalting snippets:  2 3259 3277
interpoalting snippets:  2 3284 3296
i

DONE FIXING JUMPS
current indx updated: [2630]
current indx updated: [2630]
INTERPOLATING TRACES, THRESHOLD:  200.0
current indx updated: [2621]
current indx updated: [2640]
current indx updated: [2659]
current indx updated: [2773]
current indx updated: [2849]
current indx updated: [2859]
current indx updated: [2878]
current indx updated: [2935]
current indx updated: [3667]
current indx updated: [3696]
current indx updated: [3763]
current indx updated: [3801]
current indx updated: [3953]
current indx updated: [4048]
current indx updated: [4105]
current indx updated: [4162]
current indx updated: [4210]
current indx updated: [4229]
current indx updated: [4333]
current indx updated: [4419]
current indx updated: [4438]
current indx updated: [4495]
current indx updated: [4514]
current indx updated: [4562]
current indx updated: [4571]
DELETING JUMPS FUCNTION
DONE FIXING JUMPS
current indx updated: [4571]
current indx updated: [4933]
current indx updated: [5047]
current indx updated: [5171]
c

DONE FIXING JUMPS
current indx updated: [6978]
current indx updated: [6978]
INTERPOLATING TRACES, THRESHOLD:  200.0
DELETING JUMPS FUCNTION
DONE FIXING JUMPS
current indx updated: [6978]
current indx updated: [6988]
current indx updated: [7016]
current indx updated: [7045]
current indx updated: [7064]
current indx updated: [7093]
current indx updated: [7112]
current indx updated: [7150]
current indx updated: [7159]
current indx updated: [7169]
current indx updated: [7169]
current indx updated: [7159]
current indx updated: [7197]
current indx updated: [7216]
current indx updated: [7235]
current indx updated: [7264]
current indx updated: [7292]
current indx updated: [7302]
current indx updated: [7330]
current indx updated: [7359]
current indx updated: [7387]
current indx updated: [7407]
current indx updated: [7426]
current indx updated: [7454]
current indx updated: [7483]
current indx updated: [7492]
current indx updated: [7511]
current indx updated: [7530]
current indx updated: [7540]
c

deleted small segs 3 10615
deleted small segs 3 11182
deleted small segs 3 11551
deleted small segs 3 11555
deleted small segs 3 11595
deleted small segs 3 11630
deleted small segs 3 11650
deleted small segs 3 11728
deleted small segs 3 11818
deleted small segs 3 11838
deleted small segs 3 12952
deleted small segs 3 13552
deleted small segs 3 13963
deleted small segs 3 14683
deleted small segs 3 14729
deleted small segs 3 15486
deleted small segs 3 15787
deleted small segs 3 15792
deleted small segs 3 15797
deleted small segs 3 15804
deleted small segs 3 15871
deleted small segs 3 16511
deleted small segs 3 16543
deleted small segs 3 16549
DONE DELEting triples
current indx updated: [10154]
searching... start_nan:  10214
Fast forwarding from:  10154
..... to :  10259
current indx updated: [10259]
searching... start_nan:  10318
Fast forwarding from:  10259
..... to :  10358
current indx updated: [10358]
searching... start_nan:  10389
Fast forwarding from:  10358
..... to :  10418
curren

In [14]:
print (tracesx[2311-1:2311+2,2])

[    nan 300.674     nan]


In [32]:
id_=2
for k in range(tracesx.shape[0]):
    if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
        print (k, tracesx[k,id_], tracesy[k,id_])

9318 317.532 nan
9319 317.532 nan
9320 317.532 nan
9321 317.532 nan
9322 317.532 nan
9323 317.532 nan
9324 317.532 nan
9325 317.532 nan
9326 317.532 nan
9327 317.532 nan
9328 317.532 nan
9329 317.532 nan
9789 305.184 nan


In [60]:
k = 100
id_ = 0
print (np.isnan(tracesx[k:k+3,id_]))
print (np.array([False, 
                                                 True,
                                                 False]))
print (np.all(np.isnan(tracesx[k:k+3,id_])==np.array([False, 
                                                 True,
                                                 False])))

[False False False]
[False  True False]
False
